In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import random

In [43]:
# importieren der Fund Performance Datei wo die alle funds die Preqin finden kann aufgelistet sind. Spalten enthalten u.a. die IRR und wann der Fund eröffnet wurde
df = pd.read_csv(
    "../data/raw/fund_performance.csv",
    low_memory=False  # liest alles auf einmal ein, stabilere Typbestimmung
)
df.shape

(363083, 9)

In [44]:
## Hier Fokus auf IRR

# Da von denselben Funds während der Fund Laufzeit Werte wie DPI oder RVPI dokumentiert wird, aber wir aus der Perspektive von 2025 raufschauen, nehmen wir immer nur den aktuellsten bzw. zeitlich letzten Eintrag aus der CSV. Wichtig: wir nehmen den aktuellsten Eintrag der noch eine IRR hat. Weil es sein kann dass Funds ein neues update posten aber ohne IRR, dies aber vorher mal gemacht haben. So würden wir systematisch die IRR des Funds löschen (siehe nächste Schritte)
idx = df.groupby("fund_id")["date_reported"].idxmax()

# Nur Zeilen behalten, wo net_irr_pcent existiert (nicht NaN)
df_with_irr = df[df["net_irr_pcent"].notna()]

# Innerhalb jeder fund_id den neuesten Eintrag (größtes Datum) mit IRR nehmen
idx = df_with_irr.groupby("fund_id")["date_reported"].idxmax()

# Ergebnis-DataFrame mit diesen Zeilen
df_latest = df.loc[idx].reset_index(drop=True)

In [40]:
## Hier Fokus auf Multiple

# Da von denselben Funds während der Fund Laufzeit Werte wie DPI oder RVPI dokumentiert wird, aber wir aus der Perspektive von 2025 raufschauen, nehmen wir immer nur den aktuellsten bzw. zeitlich letzten Eintrag aus der CSV 
idx = df.groupby("fund_id")["date_reported"].idxmax() 
df_latest = df.loc[idx].reset_index(drop=True)

In [45]:
# Wenn wir einen Blick draufwerfen sieht man, welche Spalten viele NaN Werte haben
df_latest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12477 entries, 0 to 12476
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   fund_id           12477 non-null  int64  
 1   date_reported     12477 non-null  object 
 2   vintage           12477 non-null  int64  
 3   called_pcent      11899 non-null  float64
 4   distr_dpi_pcent   11582 non-null  float64
 5   value_rvpi_pcent  11580 non-null  float64
 6   multiple          11384 non-null  float64
 7   net_irr_pcent     12477 non-null  object 
 8   benchmark_id      9284 non-null   float64
dtypes: float64(5), int64(2), object(2)
memory usage: 877.4+ KB


In [42]:
df_latest.head()

,fund_id,date_reported,vintage,called_pcent,distr_dpi_pcent,value_rvpi_pcent,multiple,net_irr_pcent,benchmark_id
0,2,2024-12-31,1989,97.7,179.35,0.0,1.79,NaN,15.0
1,3,2024-12-31,1993,88.5,180.74,0.0,1.81,11.40,18.0
2,4,2024-12-31,1998,99.5,75.25,0.0,0.75,-5.90,18.0
3,7,2024-12-31,1995,100.0,330.80,0.0,3.31,59.90,19.0
4,8,2024-12-31,1998,100.0,179.10,0.0,1.79,9.80,19.0


In [ ]:
12477 IRR

In [28]:
# Die Spalte mit den meisten NaN-Werten nämlich "benchmark_id" kann entfernt werden, weil wir diese nicht mehr benötigen.
df_latest = df_latest.drop(columns=["benchmark_id"])


In [30]:
# Da die IRR unsere Target Variable ist und diese nun die meisten NaN-Werte aufweist, aber dies auch nur noch weniger als 15% des Datensatzes keinen Eintrag für IRR haben, löschen wir diese Funds heraus.
df_cleaned = df_latest.dropna() # dropping Nan rows 
df_cleaned

,fund_id,date_reported,vintage,called_pcent,distr_dpi_pcent,value_rvpi_pcent,multiple,net_irr_pcent
1,3,2024-12-31,1993,88.50,180.74,0.0,1.81,11.40
2,4,2024-12-31,1998,99.50,75.25,0.0,0.75,-5.90
3,7,2024-12-31,1995,100.00,330.80,0.0,3.31,59.90
4,8,2024-12-31,1998,100.00,179.10,0.0,1.79,9.80
5,9,2024-12-31,2000,100.00,178.88,0.0,1.79,18.37
...,...,...,...,...,...,...,...,...
13505,226848,2024-09-30,2024,94.00,6.70,206.0,2.13,n/m
13506,227147,2024-09-30,2010,100.00,280.22,3.4,2.84,33.86
13507,227149,2024-09-30,2012,100.00,199.00,82.0,2.81,20.80
13508,227925,2024-12-31,2023,40.53,0.00,101.1,1.01,n/m
